In [1]:
import json
import random
import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Caricamento dei dati e creazione del modello

In [2]:
corpus = list()

In [3]:
with open ('articoli.txt', 'r', encoding = 'utf-8') as file:
    corpus = json.loads(file.read())
    
# eliminazione di quelle frasi che sono delle domande
# (per evitare che il chatbot risponda con delle domande)
for sentence in corpus:
    if (sentence[len(sentence)-1] == '?'):
        corpus.remove(sentence)

In [4]:
with open('q&a.json', 'r') as data_structure:
    q_a = json.load(data_structure)
    # appendiamo le chiavi del dizionario (le domande delle FAQ)
    for k in q_a.keys():
        corpus.append(k)

In [5]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), min_df=2, max_df = 1000)
# applichiamo il modello
matrix_tfidf = tfidf.fit_transform(corpus)

In [6]:
# funzione che calcola e restituisce la cosine similarity tra
# un vettore e la document-term matrix
def compute_similarity (model, matrix, query):
    vec_query = model.transform([query])
    # flatten () rende la similarity matrix unidimensionale
    cos_sim = cosine_similarity(vec_query, matrix).flatten()
    
    return cos_sim

In [7]:
# funzione che restituisce un saluto come risposta a un saluto dell'utente
def greeting_response(text):
    text = text.lower()
    
    # lista dei possibili saluti che il bot può dare
    bot_greetings = ['Ciao', 'Ehi']
    # lista dei saluti che il bot riconosce
    user_greetings = ['ciao']
    
    # se una parola dell'utente contiene una parola in user_greetings
    for word in text.split():
        if word in user_greetings:
            # restituisci una parola presa a caso da bot_greetings
            return random.choice(bot_greetings)

In [8]:
# funzione per riordinare una lista in ordine decrescente
def index_sort(list_var):
    length = len(list_var)
    # creo una lista di indici per la lunghezza 
    # della lista passata. Questa lista verrà 
    # e riordinata e restituita  alla fine
    list_index = list(range(0, length))
    x = list_var
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                # scambio gli elementi
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
                
    return list_index

In [9]:
def bot_response(user_input, model, matrix, q_a):
    bot_response = ''
    # flag per vedere se esiste una risposta per la query dell'utente
    response_flag = 0
    
    similarity_scores_list = compute_similarity(model, matrix, user_input)
    # index = lista di indici riodinati a seconda del similarity score
    index = index_sort(similarity_scores_list)
    
    # contatore per attaccare solo la prima risposta 
    # che abbia similarity score > 0.0
    j = 0
    for i in range(len(index)):
        if j >= 1:
            break
        # se il similarity score è maggiore di 0, 
        # significa una risposta esiste
        if similarity_scores_list[index[i]] > 0.0:
            # se la frase più simile che ha trovato è una domanda delle FAQ
            # restituisco la risposta
            if corpus[index[i]] in q_a.keys():
                bot_response = bot_response + q_a[corpus[index[i]]][0]
                response_flag = 1 # abbiamo trovato la risposta
                j = j + 1 # aggiorno il contatore per ogni risposta trovata
            # altrimenti, prende una frase degli articoli
            else:
                bot_response = bot_response + corpus[index[i]]
                response_flag = 1 # abbiamo trovato la risposta
                j = j + 1 # aggiorno il contatore per ogni risposta trovata

    # se non abbiamo trovato una risposta
    if response_flag == 0:
        bot_response = bot_response + 'Scusa, non sono in grado di risponderti.'

    return bot_response

Bot construction

In [10]:
!pip install telepot

In [12]:
# in questa cella si stabilisce il collegamento con la API del chatbot creato e si manda in ciclo infinito
#### NB: di solito il programma deve essere interrotto e la cella ri-runnata per attivare il chatbot ####
import telepot
import time

def on_chat_message(msg):
    user_input = msg["text"]
    answer = ''
    content_type, chat_type, chat_id = telepot.glance(msg)
    if content_type == 'text':
        name = msg["from"]["first_name"] # questa riga serve nel caso vogliamo recuperare il nome dell'utente e passarlo
        if greeting_response(user_input) != None:
            answer = greeting_response(user_input)
            # qui invoca la funzione chatbot response
            bot.sendMessage(chat_id, answer) # invocare la nostra funzione
        else:
            answer = bot_response(user_input, tfidf, matrix_tfidf, q_a)
            bot.sendMessage(chat_id, answer)
    else:
        bot.sendMessage(chat_id, 'Perdonami, per ora posso leggere solo testi scritti...'%name)

TOKEN = '1699693493:AAGGQgV4Wd84wOAmMqHDzf09c51osqoGk60' # il token associato a BotFather

bot = telepot.Bot(TOKEN)
bot.message_loop(on_chat_message)

while True:
    time.sleep(10) #ciclo infinito per farlo runnare in continuazione

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

...

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1181, in get_from_telegram_server
    result = self.getUpdates(offset=offset,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 1000, in getUpdates
    return self._api_request('getUpdates', _rectify(p))
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\__init__.py", line 491, in _api_request
    return api.request((self._token, method, params, files), **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 155, in request
    return _parse(r)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\telepot\api.py", line 150, in _parse
    raise exception.TelegramError(description, error_code, data)
telepot.exception.TelegramError: ('Conflict: terminated by other getUpdates request; make sure that only one bot instance is running', 409, {'ok': False, 'error_code': 409, 'description': 'Conflict: terminated by other getUpdat

KeyboardInterrupt: 